# Slimming down cleaning process and make it able to take in any new data
##### Started 14.12.22

### Imports

In [1]:
import pandas as pd
import random
import string
import re
import json
import ast
from collections import Counter


### Getting workable DF

In [2]:
data_path = '~/code/emilycardwell/final-project-data/data/raw/kaggle_raw.csv'
raw_df = pd.read_csv(data_path)
raw_df.head(3)

,Unnamed: 0,artist_name,song_name,chords&lyrics,chords,lyrics,tabs,lang,artist_id,followers,genres,popularity,name_e_chords
0,0,Justin Bieber,"10,000 Hours",\nCapo on 3rd fret\n\t \t\t \r\n\r\nVerse 1:\...,{3: 'G G/B ...,"{0: '\nCapo on 3rd fret\n\t \t\t ', 1: '', 2:...","{89: ""I-----, I'm gonna love you ""}",en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
1,1,Justin Bieber,2 Much,\n\t \t\t\r\nIntro: F#m7 D2 \r\n\r\nVerse 1:...,"{1: 'Intro: F#m7 D2 ', 4: 'F#m7 ', 8: 'D2 ', ...","{0: '\n\t \t\t', 2: '', 3: 'Verse 1:', 5: ' ...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
2,2,Justin Bieber,2u (feat. David Guetta),\n\t \t\t\r\nEm D C ...,{1: 'Em D C C...,"{0: '\n\t \t\t', 2: ""No limit in the sky that...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber


In [330]:
slim_raw_df = raw_df[['artist_name', 'song_name', 'chords', 'genres', 'popularity']]
first_raw = ast.literal_eval(slim_raw_df.loc[0, 'chords'])
first_raw

{3: 'G             G/B                         C ',
 5: '                  G ',
 7: 'G                  G/B                         C ',
 9: '                G ',
 13: '         G         Em             C       G     ',
 15: '     G        Em                C   G ',
 19: '          G            Em        C            G    ',
 21: '      G              Em                  C              G    ',
 23: '      G           Em             C         G    ',
 25: '        G            Em           C          G    ',
 27: '           G        Em       C        G            ',
 31: 'G             G/B                       C ',
 33: '                 G ',
 35: 'G                G/B                C        ',
 37: '                G ',
 41: '         G         Em             C       G     ',
 43: '     G        Em                C   G ',
 47: '          G            Em        C            G    ',
 49: '      G              Em                  C              G    ',
 51: '      G           Em       

In [325]:
def string_cleaning(df):
    split_raw_df = df.copy().loc[:1, :]
    splits = []

    for row in split_raw_df.chords:
        row_dict = ast.literal_eval(row)
        for key, val in row_dict.items():
            p1 = re.sub(r'\||\s-|-[^/]|,|\.|\*', ' ', val)
            p2 = re.sub(r'[()]', '', p1)
            s = re.sub(r'^\s+|\s+$', '', p2)
            row_dict[key] = re.sub(r'\s+', ' ', s)
        splits.append(row_dict)

    split_raw_df.chords = splits
    
    return split_raw_df.chords

In [326]:
string_cleaning(slim_raw_df)

0    {3: 'G G/B C', 5: 'G', 7: 'G G/B C', 9: 'G', 1...
1    {1: 'Intro: F#m7 D2', 4: 'F#m7', 8: 'D2', 10: ...
Name: chords, dtype: object

In [293]:
r = random.randint(0, 135782)
print(slim_raw_df.chords[r])
print(split_raw_df.chords[0])

{1: 'Intro ', 2: 'C  C  F  F  C  C7  F  F ', 3: 'C  C  F  F  C  C7  F  F ', 4: 'C ', 7: 'C    F        C ', 9: '    C        F      C ', 11: '       C     F         C ', 13: 'Fm       C ', 17: '   C     F           C ', 19: '      C       F        C ', 21: '         C       F        C ', 23: 'Fm        C ', 26: '                C              F ', 28: 'C             C7 ', 30: '        F ', 32: '          C                          F ', 34: 'C             C7 ', 36: '          F ', 38: '               C ', 41: 'Intro ', 42: 'C  F  C  Csus4  C ', 43: 'C  F  C  Csus4  C ', 44: 'C  F  C  Csus4  C ', 45: 'Fm C ', 47: '                C              F ', 49: 'C             C7 ', 51: '        F ', 53: '          C                          F ', 55: 'C             C7 ', 57: '          F ', 59: '               C ', 63: '   C    F              C ', 65: '      C       F        C ', 67: '         C       F        C ', 69: 'Fm       C ', 73: 'Fm ', 75: 'C            Csus4  C '}
{3: 'G G/B C', 5: 'G',

#### See Chord Frequency

In [6]:
chords_counter = Counter()
for song in splits:
    for value in song.values():
        l = value.split(' ')
        chords_counter.update(l)
del chords_counter['']

chord_count_df = pd.Series(dict(chords_counter)).to_frame('chord_count')
sorted_cc_df = chord_count_df.sort_values(by='chord_count', ascending=False)
len(sorted_cc_df)

25637

In [7]:
letters = list(string.ascii_uppercase)[:7]
non_chord_filter = [chord for chord in sorted_cc_df.index if chord[0] in letters]
cc_df = sorted_cc_df[sorted_cc_df.index.isin(non_chord_filter)]

# pd.set_option('display.max_rows', 1190)
## sorted_cc_df.head(70*17)
pd.set_option('display.max_rows', 100)
cc_df.iloc[100:200]

,chord_count
F7+,5598
Bbm7,5490
Gmaj7,5407
C7+,5372
G7+,5316
Amaj7,5227
C#5,5177
Bb9,5056
A7+,4993
A7/13,4972


In [8]:
# slim_chord_counts_dict = {}
# for chord, count in chords_count_dict.items():
#     if count <= low_freq_to_remove:
#         pass
#     else:
#         slim_chord_counts_dict[chord] = count

## Creating List of Correctly Formatted Chords

In [9]:
major_chords = ['C', 'C#', 'Db', 'D', 'D#', 'Eb', 'E', 'F', 'F#', 'Gb', 'G', 'G#', 'Ab', 'A', 'A#', 'Bb', 'B']

In [10]:
chords_df = pd.DataFrame(index=['', 'm', 'dim', 'aug', '5', 'aug5', '5b7',
                                'dim7', 'hdim7', 'm7', 'm-M7', '7', 'aug7', 'M7',  
                                'm9', '9', '7b9',  'M9', 'm11', '11',
                                'add2', 'add4', 'add6', 'add9', 
                                'madd2', 'madd4', 'madd6', 'madd9', 
                                '7add4', '7addb6', '7add6', 
                                'msus2', 'msus4', '7sus2', '7sus4', 'sus2', 'sus4'
                                ],
                         columns=major_chords)


In [11]:
for idx in chords_df.index:
    new_chords = [m + idx for m in major_chords]
    chords_df.loc[idx, :] = new_chords
chords_df.rename(index={'': 'major'}, inplace=True)
chords_df.head(3)

,C,C#,Db,D,D#,Eb,E,F,F#,Gb,G,G#,Ab,A,A#,Bb,B
major,C,C#,Db,D,D#,Eb,E,F,F#,Gb,G,G#,Ab,A,A#,Bb,B
m,Cm,C#m,Dbm,Dm,D#m,Ebm,Em,Fm,F#m,Gbm,Gm,G#m,Abm,Am,A#m,Bbm,Bm
dim,Cdim,C#dim,Dbdim,Ddim,D#dim,Ebdim,Edim,Fdim,F#dim,Gbdim,Gdim,G#dim,Abdim,Adim,A#dim,Bbdim,Bdim


In [12]:
sharp_notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B', 
                'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
flats_notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B', 
                'C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
sharp_keys = ['D', 'E', 'G', 'A', 'B', 'Dm', 'Em', 'Fm', 'Bm']
flats_keys = ['C', 'F', 'Cm', 'Gm', 'Am']

slash_indexes = ['dim/b7', 'm/2', 'm/b3', 'm/3', 'm/4', 'm/5', 'm/6','m/b7', 'm/7',
                'm7/b3', 'm7/4', 'm7/b5', 'm7/5', 'm7/7', 
                '/2', '/b3', '/3', '/4', '/5', '/6','/b7', '/7', 
                '7/b2', '7/2', '7/b3', '7/3', '7/4', '7/5', '7/b6', '7/6','7/7', 
                '9/b3', '9/3', '9/4', '9/5', '9/6', '9/7',
                'M7/2', 'M7/b3', 'M7/3', 'M7/4', 'M7/5', 'M7/6','M7/b7']

whole_to_half = {'b2': 1, '2': 2, 'b3': 3, '3': 4, '4': 5, 'b5': 6,
                 '5': 7, 'b6': 8,'6': 9, 'b7': 10, '7': 11}

In [13]:
# big function
def get_slash_notes(slash_indexes, columns):
    slash_chord = pd.DataFrame(index=slash_indexes, columns=columns)
    i = 0 
    
    def find_slash(chord, notes, i):
        for si in slash_indexes:
            s = si.split('/')
            h = whole_to_half[s[1]]
            sc = f'{chord}{s[0]}/{notes[i+h]}'
            slash_chord.loc[si, chord] = sc
    
    def sort_f_s(chord, notes):
        for n in notes:
            if len(chord) > 1:
                i = notes.index(chord)
                find_slash(chord, notes, i)
            else:
                i = notes.index(chord[0])
                find_slash(chord, notes, i)
    
    for chord in columns:
        if chord in sharp_keys or '#' in chord:
            sort_f_s(chord, sharp_notes)
        else:
            sort_f_s(chord, flats_notes)
                
            
    return slash_chord

In [14]:
slash_chords_df = pd.concat([chords_df, get_slash_notes(slash_indexes, chords_df.columns)])
slash_chords_df

,C,C#,Db,D,D#,Eb,E,F,F#,Gb,G,G#,Ab,A,A#,Bb,B
major,C,C#,Db,D,D#,Eb,E,F,F#,Gb,G,G#,Ab,A,A#,Bb,B
m,Cm,C#m,Dbm,Dm,D#m,Ebm,Em,Fm,F#m,Gbm,Gm,G#m,Abm,Am,A#m,Bbm,Bm
dim,Cdim,C#dim,Dbdim,Ddim,D#dim,Ebdim,Edim,Fdim,F#dim,Gbdim,Gdim,G#dim,Abdim,Adim,A#dim,Bbdim,Bdim
aug,Caug,C#aug,Dbaug,Daug,D#aug,Ebaug,Eaug,Faug,F#aug,Gbaug,Gaug,G#aug,Abaug,Aaug,A#aug,Bbaug,Baug
5,C5,C#5,Db5,D5,D#5,Eb5,E5,F5,F#5,Gb5,G5,G#5,Ab5,A5,A#5,Bb5,B5
aug5,Caug5,C#aug5,Dbaug5,Daug5,D#aug5,Ebaug5,Eaug5,Faug5,F#aug5,Gbaug5,Gaug5,G#aug5,Abaug5,Aaug5,A#aug5,Bbaug5,Baug5
5b7,C5b7,C#5b7,Db5b7,D5b7,D#5b7,Eb5b7,E5b7,F5b7,F#5b7,Gb5b7,G5b7,G#5b7,Ab5b7,A5b7,A#5b7,Bb5b7,B5b7
dim7,Cdim7,C#dim7,Dbdim7,Ddim7,D#dim7,Ebdim7,Edim7,Fdim7,F#dim7,Gbdim7,Gdim7,G#dim7,Abdim7,Adim7,A#dim7,Bbdim7,Bdim7
hdim7,Chdim7,C#hdim7,Dbhdim7,Dhdim7,D#hdim7,Ebhdim7,Ehdim7,Fhdim7,F#hdim7,Gbhdim7,Ghdim7,G#hdim7,Abhdim7,Ahdim7,A#hdim7,Bbhdim7,Bhdim7
m7,Cm7,C#m7,Dbm7,Dm7,D#m7,Ebm7,Em7,Fm7,F#m7,Gbm7,Gm7,G#m7,Abm7,Am7,A#m7,Bbm7,Bm7


## Shove chords into correct format

In [222]:
substitutions = {'maj': '', ',': '', 'º': 'dim', '5+': 'aug', '7M': 'M7',
                 'maj7': 'M7', 'm7b5': 'hdim7', 'm7+': 'm-M7', '7+': 'aug7', 
                 '79': '9', 'm5-/7': 'dim/b7', '711': '7add4', '7b13': '7addb6', 
                 '713': '7add6', '4/7/9': '9add4', '69': '9add6', '/9-': '/b2', 
                 '/9': '/2', '/5-': '/b5', '/13-': '/b6', '/5+': '/b6', '/13': '/6',
                 '2': 'add2', '4': 'add4', '6': 'add6', 'sus': 'sus4'
                 }

In [290]:
def merge_chords(chords_column):
    chords_column_copy = chords_column.copy()
    merged_chords = []
    rejected = []
    added = []
    
    # compare against df
    def counters(chord):
        if chord in slash_chords_df.columns:
            added.append(chord)
        elif chord in slash_chords_df.values:
            print(chord, 'in_df')
            added.append(chord)
        else:
            print(chord, 'womp')
            rejected.append(f'{chord}-f')
        return 
    
    # if keys in subs in chord, replace with values
    def check_subs(chord):
        for k, s in substitutions.items():
            if k in chord: 
                print(k, chord)
                ch_sub = chord.replace(k, s)
                print(ch_sub, 'subbed')
                return ch_sub
            else:
                return chord
    
    for song in chords_column_copy:
        merged_song = {}
        for m, chords in song.items():
            measure = chords.split()
            for idx, ch in enumerate(measure):
                # replace repetition symbols with previous chord
                if idx == 0 and m == :
                    rep = re.sub(r'%|2x|x2', song[m-1][-1], ch)
                    chs = check_subs(ch)
                    counters(chs)
                    measure[idx] = chs
                else:
                    rep = re.sub(r'%|2x|x2', measure[idx-1], ch)
                    chs = check_subs(rep)
                    counters(chs)
                    measure[idx] = chs                
            merged_song[m] = measure
        merged_chords.append(merged_song)
    
    added_c = Counter(added)
    rejects_c = Counter(rejected)
    
    added_df = pd.Series(dict(added_c)).to_frame('added').sort_values(by='added', ascending=False)
    rejects_df = pd.Series(dict(rejects_c)).to_frame('rejects').sort_values(by='rejects', ascending=False)
 
    return pd.DataFrame(merged_chords).T, added_df, rejects_df

In [295]:
# merged, rejects, adds = merge_chords(split_raw_df.chords[0])
split_raw_df.chords[0]

{3: 'G G/B C',
 5: 'G',
 7: 'G G/B C',
 9: 'G',
 13: 'G Em C G',
 15: 'G Em C G',
 19: 'G Em C G',
 21: 'G Em C G',
 23: 'G Em C G',
 25: 'G Em C G',
 27: 'G Em C G',
 31: 'G G/B C',
 33: 'G',
 35: 'G G/B C',
 37: 'G',
 41: 'G Em C G',
 43: 'G Em C G',
 47: 'G Em C G',
 49: 'G Em C G',
 51: 'G Em C G',
 53: 'G Em C G',
 55: 'G Em C G',
 57: 'G Em C G',
 60: 'Bridge:',
 61: 'Bm C',
 63: 'Bm C',
 69: 'G Em C G',
 71: 'G Em C G',
 73: 'G Em C G',
 75: 'G Em C G',
 77: 'G Em C G',
 79: 'G Em C G',
 84: 'G Em C',
 86: 'C G G',
 88: 'G Em C G'}

#### For next time:
- fix rejects (reject) tag

In [49]:
rejects_counter = Counter()
for r in rejects:
    for value in song.values():
        l = value.split(' ')
        chords_counter.update(l)

rejects_df = pd.Series(dict(rejects_counter)).to_frame('rejects')
sorted_rej_df = rejects_df.sort_values(by='rejects', ascending=False)
sorted_rej_df

TypeError: unhashable type: 'list'

In [ ]:
# # big function
# def translate_slashes(slash_indexes, columns):
#     slash_chord = pd.DataFrame(index=slash_indexes, columns=columns)
#     i = 0 
    
#     def find_slash(chord, notes, i):
#         for si in slash_indexes:
#             s = si.split('/')
#             h = whole_to_half[s[1]]
#             sc = f'{chord}{s[0]}/{notes[i+h]}'
#             slash_chord.loc[si, chord] = sc
    
#     def sort_f_s(chord, notes):
#         for n in notes:
#             if n in chord :
#                 i = notes.index(chord[0])
#                 find_slash(chord, notes, i)
#             elif len(chord) > 1:
#                 if '#' == chord[1]:
#                     i = sharp_notes.index(chord[:2])
#                     find_slash(chord, sharp_notes, i)
#                 elif 'b' == chord[1]:
#                     i = flats_notes.index(chord[:2])
#                     find_slash(chord, flats_notes, i)
    
#     for chord in columns:
#         sort_f_s(chord, sharp_notes)
#         sort_f_s(chord, flats_notes)
                
            
#     return slash_chord

In [ ]:
# slash_chords_df = pd.concat([chords_df, get_slash_notes(slash_indexes, chords_df.columns)])
